###### 2021-03-28

# Co-Routine (코루틴)
- 코루틴을 알아보기 전에 *메인함수*와 *서브함수*를 알아보자.

In [1]:
def sub_function(a, b):
    c = a + b
    print(c)
    print('서브 함수')

def main_function():
    sub_function(1, 2)
    print('메인 함수')

main_function()

3
서브 함수
메인 함수


![041001](https://user-images.githubusercontent.com/54063179/112755008-d5c56e00-9019-11eb-9ac1-df59eba60974.png)
- 위 그림처럼 메인함수와 서브함수의 관계에서는 `메인함수` > `서브함수` > `메인함수`로 코드를 실행한다.
- 또한, `서브함수`를 실행하고 다시 `메인함수`로 돌아오면, `서브함수`의 내용은 모두 사라지게 된다.

![041002](https://user-images.githubusercontent.com/54063179/112755090-276df880-901a-11eb-9887-cb49a87658df.png)
- 반면에 코루틴은 *Copoerative routine*의 줄임말로 서로 협력하는 루틴이다.
- 위에서 예시로 들었던 `메인함수`와 `서브함수`처럼 종속의 관계가 아니라 *서로 대등한 관계*이다.
- 그러므로 코루틴은 함수가 종료되지 않은 상태에서 메인 루틴의 코드를 실행한 뒤 다시 원래의 코드로 돌아온다. 이때, 코루틴이 종료되어도 코루틴의 내용이 사라지지 않는다.

## 1. 코루틴에 값 보내기
- 코루틴은 제너레이터의 한 종류이다.
- 제너레이터는 `yield`로 값을 발생시켰지만, 코루틴은 `yield`로 값을 받아올 수 있습니다.
- 다음과 같이 코루틴을 실행할때는 `send`메서드를 사용한다.
    - `코루틴객체.send(값)`
    - `변수 = (yield)`

In [2]:
def number_generator():
    while True:  # 코루틴을 무한히 사용하기위해 While True
        x = (yield)  # 코루틴 밖에서 send메서드를 이용해 값을 받아옴
        print(x)


co = number_generator()
next(co)  # 코루틴 안에 `yield` 까지 실행
          # 코루틴을 최초로 실행 하는 것이라고 보면됨
          # `co.send(None)`를 대신 사용해도 된다.

co.send(1)  # 코루틴에 숫자 1 보냄
co.send(2)  # 코루틴에 숫자 2 보냄
co.send(3)  # 코루틴에 숫자 3 보냄

1
2
3


- 아래는 위 코드의 실행 과정을 도식화한 것이다.
![041003](https://user-images.githubusercontent.com/54063179/112755406-808a5c00-901b-11eb-818a-55edb6972a2d.png)

## 2. 코루틴 밖으로 보내기
- 이제는 코루틴을 밖으로 보내보자.
- 다음과 같이 (yield 변수) 형태로 yield에 변수를 지정한 뒤 괄호로 묶어주면 값을 받아오면서 바깥으로 값을 전달한다.
- 그리고 받음과 동시에 값을 밖으로 전달한다.

In [4]:
def sum_coroutine():
    total = 0
    while True:
        x = (yield total)  # 밖에서 받아오는 값은 x에 밖으로 보낼 값은 total
        total += x


co = sum_coroutine()

print(co.send(None))  # 코루틴 최초 실행 next(co)로도 실행가능
print(co.send(1))
print(co.send(2))
print(co.send(3))


0
1
3
6


![041005](https://user-images.githubusercontent.com/54063179/112844091-49c94a00-90de-11eb-97ae-a85eccbfd2ff.png)

## 3. 코루틴 강제 종료하기
- 보통 코루틴은 실행 상태를 유지하기 위해 `while True:`와 함께 사용된다.
- 만약 이러한 코루틴을 강제 종료하고 싶다면, `close` 메소드를 사용합니다.
- 아래의 경우는 `close`를 사용하지 않아도 멈추는 코드이지만, `close`는 코루틴의 종료 시점을 알아야할때, 사용하면 편리하다.

In [11]:
def number_coroutine():
    while True:
        x = (yield)
        print(x, end=' ')

co = number_coroutine()

next(co)

for i in range(20):
    co.send(i)

co.close()

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 

## 4. GeneratorExit 처리하기
- 코루틴 객체에서 close 메소드를 호출하면 코루틴이 종료될때 GeneratorExit 예외가 발생한다.
- 이 예외 시점을 사용하면 코루틴의 종료시점을 알 수 있다.

In [12]:
def number_generator():
    try:
        while True:
            x = (yield)
            print(x, end=' ')
    except GeneratorExit:
        print()
        print('코루틴 종료')


co = number_generator()
next(co)

for i in range(20):
    co.send(i)

co.close()

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
코루틴 종료


## 5. 코루틴 안에서 예외 발생시키기
- 외부에서 코루틴 안에 특정 예외를 발생시켜 코루틴을 종료해보자
- 코루틴 안에 예외를 발생 시킬 때에는, `throw`메소드를 사용한다.
- `throw`는 말 그대로 예외을 코루틴안으로 던지다는 뜻이다.
- 이때 `throw`메소드에 지정한 에러 메세지는 except as의 변수에 들어간다.
- `코루틴객체.throw(예외이름, 에러메시지)`

In [15]:
def sum_coroutine():
    total = 0
    try:
        while True:
            x = (yield)
            total += x
    except RuntimeError as e:
        print(e)
        yield total


co = sum_coroutine()
next(co)

for i in range(20):
    co.send(i)

print(co.throw(RuntimeError, '예외로 코루틴 끝내기'))

예외로 코루틴 끝내기
190


## 6. 하위 코루틴의 반환값 가져오기
- 제너레니어테서 `yield from`을 사용하면, 값을 바깥으로 여러 번 전달한다.
- 하지만 코루틴에서는 조금 다르게 적용한다.
- `yield from`에 코루틴을 지정하면 해당 코루틴에서 return으로 반환값을 가져온다.
- `변수 = yiled from 코루틴()`

In [2]:
def accunulate():
    total = 0
    while True:
        x = (yield)
        if x is None:
            return total
        total += x

def sum_coroutine():
    while True:
        total = yield from accunulate()  # 상위 코루틴에서 yield from을 시용하면 send 메소드를 이용해 하위 코루틴에까지 값을 전달할 수 있다.
        print(total)

co = sum_coroutine()
next(co)

for i in range(1, 11):
    co.send(i)
co.send(None)

for i in range(1, 101):
    co.send(i)
co.send(None)


55
5050


## 7. 코루틴의 yield from 으로 값을 발생 시키기
- 이전 까지는 `x = (yield)`와 같이 코루틴 바깥에서 보낸 값만 받아왔다.
- 음.. 이부분은 아무리 봐도 이해가 가지않습니다..

In [3]:
def number_coroutine():
    x = None
    while True:
        x = (yield x)  # 코루틴 바깥에서 값을 받아오면서 바깥으로 값을 전달
        if x == 3:
            return x

def print_coroutine():
    while True:
        x = yield from number_coroutine()
        print(f'print coroutine : {x}')



co = print_coroutine()
next(co)

x = co.send(1)
print(x)
x = co.send(2)
print(x)
x = co.send(3)
print(x)

1
2
print coroutine : 3
None
